LOAD DATASET

In [2]:
import pandas as pd
import numpy as np

data= pd.read_csv("dataset.csv")
data

,Text,Label
0,Damn I thought they had strict gun laws in Ger...,0
1,I dont care about what it stands for or anythi...,0
2,It's not a group it's an idea lol,0
3,So it's not just America!,0
4,The dog is a spectacular dancer considering he...,0
...,...,...
17591,I find rats nicer and cleaner than most Chinese,1
17592,"Check out this niggar, they hit things like wi...",1
17593,"this country has become an absolute shambles, ...",0
17594,Me aged 16 = anti-Semitism is bad Me aged 18 =...,1


CLEANING

In [3]:
import re
def data_cleaning(text):

  text = re.sub(r'\s+', ' ', text) #the re.sub function replaces one or more whitespace characters (\s+) with a single space.

  text = re.sub(r"@\S+", "",text)

  text = re.sub(r'#', '', text)

  import contractions
  text=contractions.fix(text)

  text = text.lower()

  text = re.sub(r'[^\w\s]', '', text)

  text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

  text = ' '.join([word for word in text.split() if len(word) > 2 or word.isnumeric()])

  from nltk.stem import WordNetLemmatizer
  lemmatizer = WordNetLemmatizer()
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

  return text

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
data['Text']=data['Text'].apply(data_cleaning)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Split for train and test

In [4]:
from sklearn.model_selection import train_test_split

X=data['Text']
y=data['Label']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Encode label column with label encoder

In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
y_val = label_encoder.transform(y_val)

 Tokenization using keras tokenizer

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)

X_train_pad = pad_sequences(X_train_seq, maxlen=70,padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=70,padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=70,padding='post')

X_train_pad

array([[   8,   88,  207, ...,    0,    0,    0],
       [ 211,  157,  831, ...,    0,    0,    0],
       [ 464,    3,    4, ...,    0,    0,    0],
       ...,
       [   1,    0,    0, ...,    0,    0,    0],
       [ 174,    3, 8209, ...,    0,    0,    0],
       [  10,  886, 1226, ...,    0,    0,    0]])

EMBEDDING

In [ ]:
texts = data['Text'].values
labels = data['Label'].values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 

def one_hot_encoding(texts):
    vectorizer = CountVectorizer(binary=True)
    embeddings = vectorizer.fit_transform(texts)
    return embeddings

embeddings_one_hot = one_hot_encoding(texts)
X_train_one_hot, X_test_one_hot, y_train_one_hot, y_test_one_hot = train_test_split(embeddings_one_hot, labels, test_size=0.2, random_state=42)
print(embeddings_one_hot)

In [ ]:
def term_frequency_encoding(texts):
    vectorizer = CountVectorizer()
    embeddings = vectorizer.fit_transform(texts)
    return embeddings
embeddings_tf = term_frequency_encoding(texts)
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(embeddings_tf, labels, test_size=0.2, random_state=42)
print(embeddings_tf)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_embedding(texts):
    vectorizer = TfidfVectorizer()
    embeddings = vectorizer.fit_transform(texts)
    return embeddings

embeddings_tfidf = tfidf_embedding(texts)
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(embeddings_tfidf, labels, test_size=0.2, random_state=42)


print(embeddings_tfidf)
print(embeddings_tfidf.shape)

BALANCING

In [7]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_train_pad, y_train = smote.fit_resample(X_train_pad,y_train)

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd
data = pd.read_csv("Embedded_data.csv")
X=data.drop(columns=['Label','Text','Tokens'])
y=data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

MODEL EVALUATION

In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd
data = pd.read_csv("Embedded_data.csv")
X=data.drop(columns=['Label','Text','Tokens'])
y=data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score,accuracy_score,confusion_matrix,roc_auc_score

# Define the hyperparameter grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
    'solver': ['liblinear', 'lbfgs']
}

# Initialize GridSearchCV
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy')

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate the classifier
print("Best Logistic Regression using Grid Search")
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

Best Logistic Regression using Grid Search
Precision: 0.639018691588785
Recall: 0.38359046283309955
Accuracy: 0.6625
ROC-AUC Score: 0.6180129964595297
Confusion Matrix:
 [[1785  309]
 [ 879  547]]


TUNNING

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

data= pd.read_csv("Embedded_data.csv")


X=data.drop(columns=['Label','Text','Tokens'])# Features
y = data['Label']# Features

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the Logistic Regression classifier
logreg = LogisticRegression(max_iter=1000)

# Define the grid of hyperparameters to search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l1', 'l2'],  # Regularization penalty
    'solver': ['liblinear', 'saga']  # Optimization algorithm
}

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# Print best hyperparameters and best score
print("Best hyperparameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy:", test_accuracy)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means

Best hyperparameters: {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Best cross-validation accuracy: 0.6683454740549747
Test set accuracy: 0.6673612426595946


DL MODEL

In [7]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file='glove.6B.200d.txt'
word2vec_output_file='word2vec.txt'
glove2word2vec(glove_input_file,word2vec_output_file)

<ipython-input-7-62d0641268e2>:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file,word2vec_output_file)


(400000, 200)

In [8]:
from gensim.models import KeyedVectors
model= KeyedVectors.load_word2vec_format(word2vec_output_file)

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

# Load the dataset
data = pd.read_csv('dataset.csv')

# Preprocess the data
texts = data['Text'].values
labels = data['Label'].values

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Tokenize the text
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X, maxlen=200)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [11]:
def glove_embeddings(filepath, word_index, embedding_dim):
    embeddings_index = {}
    with open(filepath, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

embedding_dim = 200
glove_filepath = 'glove.6B.200d.txt'
embedding_matrix = glove_embeddings(glove_filepath, tokenizer.word_index, embedding_dim)

In [14]:
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Activation
# Build the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=200))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.4))
model.add(Dense(256,activation='sigmoid'))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64,callbacks=[early_stopping])

Epoch 1/10
220/220 [==============================] - 1997s 9s/step - loss: 0.6414 - accuracy: 0.6252
Epoch 2/10
220/220 [==============================] - 1991s 9s/step - loss: 0.5572 - accuracy: 0.7111
Epoch 3/10
220/220 [==============================] - 2005s 9s/step - loss: 0.4971 - accuracy: 0.7594
Epoch 4/10
220/220 [==============================] - 1998s 9s/step - loss: 0.4208 - accuracy: 0.8095
Epoch 5/10
220/220 [==============================] - 1997s 9s/step - loss: 0.3418 - accuracy: 0.8553
Epoch 6/10
220/220 [==============================] - 1995s 9s/step - loss: 0.2690 - accuracy: 0.8927
Epoch 7/10
220/220 [==============================] - 1999s 9s/step - loss: 0.2053 - accuracy: 0.9214
Epoch 8/10
220/220 [==============================] - 1944s 9s/step - loss: 0.1600 - accuracy: 0.9429
Epoch 9/10
220/220 [==============================] - 1951s 9s/step - loss: 0.1324 - accuracy: 0.9562
Epoch 10/10
220/220 [==============================] - 1946s 9s/step - loss: 0.107

In [15]:
from sklearn.metrics import classification_report
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")
y_pred = y_pred.flatten()
print(classification_report(y_test,y_pred))

110/110 [==============================] - 177s 2s/step
              precision    recall  f1-score   support

           0       0.74      0.74      0.74      2094
           1       0.62      0.62      0.62      1426

    accuracy                           0.69      3520
   macro avg       0.68      0.68      0.68      3520
weighted avg       0.69      0.69      0.69      3520

